## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Medical LLM - Medium](https://aws.amazon.com/marketplace/pp/prodview-z4jqmczvwgtby)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.


- **Model**: `jsl-medical-summarization-qa-70b`
- **Model Description**: Medical model for summarization, question answering (open-book and closed-book), and general chat.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import os
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
model_name = "jsl-medical-summarization-qa-70b"

real_time_inference_instance_type = "ml.g6.48xlarge"
batch_transform_inference_instance_type = "ml.g5.48xlarge"

## 2. create a deployable model from the model package.

In [4]:
# Define ModelPackage
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session, 
)

### Important Parameters

**Required Parameter:**
- **`input_text`**: The input text(s) provided to the model.
  - **Type**: `Union[str, dict, List[str], List[dict]]`
  - **Constraints**: Provide a string for a single text or a list of strings for multiple inputs. Additionally, you can also provide a dictionary or a list containing multiple dictionaries with keys `context` and `question`. If you select the `open_book_qa` template, the model will format the input accordingly. If no template is selected, the `context` and `question` strings will be concatenated with a newline.

**Optional Parameters for JSON Input Format:**
- **`max_new_tokens`**: The maximum number of tokens the model should generate as output. (*Can be passed as an environment variable when initializing the SageMaker Transformer object for batch inference.*)
  - **Type**: `int`
  - **Default**: `1024`
  - **Constraints**: Must be a positive integer greater than 0.

- **`temperature`**: The temperature parameter controlling the randomness of token generation. (*Can be passed as an environment variable when initializing the SageMaker Transformer object for batch inference.*)
  - **Type**: `float`
  - **Default**: `0.2`
  - **Constraints**: Must be a float between 0.0 and 1.0.

- **`repetition_penalty`**: The repetition penalty parameter that penalizes new tokens based on whether they appear in the prompt and the generated text so far. Values > 1 encourage the model to use new tokens, while values < 1 encourage the model to repeat tokens. (*Can be passed as an environment variable when initializing the SageMaker Transformer object for batch inference.*)
  - **Type**: `float`
  - **Default**: `1.0`
  - **Constraints**: Must be a float greater than 0 and less than or equal to 2.
  
- **`template`**: You can select the predefined template.
  - **Type**: `str`
  - **Default**: `None`
  

You can pick one of the following templates:
```json
{
    "summarization": "Summarize the following document:\n## Document Start ##\n{context}\n## Document End ##",
    "open_book_qa": "Answer the following question based on the given context:\n## Context Start ##\n{context}\n## Context End ##\n## Question Start ##\n{question}\n## Question End ##",
    "closed_book_qa": "Answer the following question:\n## Question Start ##\n{question}\n## Question End ##",
}
```
You can select any of these templates according to your use case. We perform string formatting, then pass it to the chat template.

If no template is provided, we take your `input_text` as it is and do not perform any formatting, directly passing your input to the chat template.

> **Parameter Priority**: User-provided parameters are given priority, followed by environment variables, and finally default values.

---

### Model Configuration

| Parameter                  | Value     | Description                                                                                                                                                                                                                                                                                                                               |
|----------------------------|-----------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`dtype`**                | `float16` | The data type for the model weights and activations.                                                                                                                                                                                                                                                                                      |
| **`max_model_len`**        | `8192`  | This indicates that your input and the model response combined should come under this limit (`input + output <= max_model_len`). |
| **`tensor_parallel_size`** | Variable  | The number of GPUs to use for distributed execution with tensor parallelism.                                                                                                                                                                                                                                                              |

Other than the parameters mentioned above, we are utilizing the default parameters specified for the `LLM` class in the [VLLM documentation](https://docs.vllm.ai/en/latest/dev/offline_inference/llm.html).

#### Instance-Specific `max_model_len` Values

| Instance Type       | Inference Type |Number of GPUs | Total GPU Memory (GB) | max_model_len   |
|---------------------|----------------|---------------|-----------------------|-----------------| 
| `ml.g6.48xlarge`    |     Real-Time  |8              | 192                   | 8,192           |
| `ml.g5.48xlarge`    |     Batch      |8              | 192                   | 8,192           |


Total Memory values are approximate. Usable memory may be slightly less. For pricing details, visit the [Amazon SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/).



### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### Format 1: Single Input

Provide a single input as a JSON object.

```json
{
    "input_text": "input text"
}
```

Alternatively, you can provide a dictionary containing the keys `context` and `question`.

```json
{
    "input_text": {"context": "Context", "question": "Question"},
    "template": "open_book_qa"
}
```

#### Format 2: Multiple Inputs

Provide multiple inputs as a JSON array.

```json
{
    "input_text": [
        "input text 1",
        "input text 2"
    ]
}
```

Alternatively, you can provide an array of dictionaries, where each dictionary contains the keys `context` and `question`.

```json
{
    "input_text": [
        {"context": "Context 1", "question": "Question 1"},
        {"context": "Context 2", "question": "Question 2"}
    ],
    "template": "open_book_qa"
}
```

**Note**: If you provide dictionaries, you can specify the template as `open_book_qa`. If you don't select any template, it will concatenate both strings with a newline.


#### Format 3: JSON Lines (JSONL):

Provide input in JSON Lines format, where each line is a JSON object.
```
{"input_text": "input text 1"}
{"input_text": "input text 2"}
```


## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [5]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
    model_data_download_timeout=3600
)

---------------------------------------!

In [6]:
import time

def invoke_realtime_endpoint(record):

    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(record),
    )

    return json.load(response["Body"])

### Question Answering

#### Initial setup

In [7]:
open_book_qa = [
    {
        "context": "Within the last several days, he has seen you in clinic and once again discussed whether or not to undergo radiation for his left temporal lesion. The patient has clearly been extremely ambivalent about this therapy for reasons that are not immediately apparent. It is clear that his MRI is progressing and that it seems unlikely at this time that anything other than radiation would be particularly effective. Despite repeatedly emphasizing this; however, the patient still is worried about potential long-term side effects from treatment that frankly seem unwarranted at this particular time",
        "question": "What is the patient's concern regarding undergoing radiation therapy for his left temporal lesion?"
    },
    {
        "context": "The patient is a 52-year-old male who is here for independent medical evaluation. The patient states that he works for ABC ABC as a temporary worker. He worked for ABCD too. The patient's main job was loading and unloading furniture and appliances for the home. The patient was approximately there for about two and a half weeks. Date of injury occurred back in October. The patient stating that he had history of previous back problems ongoing; however, he states that on this particular day back in October, he was unloading an 18-wheeler at ABC and he was bending down picking up boxes to unload and load. Unfortunately at this particular event, the patient had sharp pain in his lower back. Soon afterwards, he had radiating symptoms down his right buttock all the way down to the lateral part of his leg crossing his knee. This became progressively worse", 
        "question": "What symptoms did the patient experience after the injury?"
    },
    {
        "context": "Patient with a past medical history of hypertension for 15 years. (Medical Transcription Sample Report) HISTORY OF PRESENT ILLNESS: The patient is a 74-year-old white woman who has a past medical history of hypertension for 15 years, history of CVA with no residual hemiparesis and uterine cancer with pulmonary metastases, who presented for evaluation of recent worsening of the hypertension. According to the patient, she had stable blood pressure for the past 12-15 years on 10 mg of lisinopril.", 
        "question": "What is the primary issue reported by the patient?"
    }
]


closed_book_qa = """
A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.

Which of the following is the best treatment for this patient?
A: Ampicillin
B: Ceftriaxone
C: Ciprofloxacin
D: Doxycycline
E: Nitrofurantoin
"""


#### Example: 1

In [8]:
input_data = {
    "input_text": closed_book_qa,
    "template": "closed_book_qa"
}

data  = invoke_realtime_endpoint(input_data)
pd.DataFrame(data)

,response
0,The best answer is E.


#### Example: 2

In [10]:
input_data = {
    "input_text": open_book_qa,
    "template": "open_book_qa"
}
# The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.
data = invoke_realtime_endpoint(input_data)
pd.DataFrame(data)

,response
0,The patient is worried about potential long-term side effects from radiation therapy.
1,"The patient experienced sharp pain in his lower back, followed by radiating symptoms down his right buttock, all the way down to the lateral part of his leg, crossing his knee."
2,The primary issue reported by the patient is the recent worsening of her hypertension.


### Summarization

#### Initial setup

In [12]:
# Articles related to COVID-19 from the CORD-19 dataset
covid19_articles = [
    """INTRODUCTION: Infection with the new coronavirus responsible for Severe Acute Respiratory Syndrome (SARS-CoV-2) continues to spread worldwide. In Brazil, there are already more than 230 thousand dead, many of these older adults. OBJECTIVE: To present the clinical characteristics of older Brazilian adults infected by COVID-19, in the epidemiological weeks (EW) 34-52, and to verify factors responsible for the increased risk of death. METHODS: Retrospective and observational study conducted with secondary publicly available data, provided by the Brazilian Ministry of Health. 1,544 confirmed cases of registered COVID-19 infection were included between August 16 and December 26, 2020, aged 60 or older. OUTCOMES: Demographic data, comorbidity, symptoms for disease, clinical information: days of hospitalization, chest X-ray, type of RT-PCR. RESULTS: 48% of patients admitted to the ICU with evidence for SARS-CoV-2 died. Symptoms and comorbidities related to increased chance of death (OR) were immunodeficiency (188%), kidney disease (166%), neurological disease (103%), dyspnea (86%), pneumopathy (55%), O2 saturation <95% (53%), respiratory discomfort (49%), age (36%), sore throat (31%), and sex (0.5%). There was a 5% increase in the chance of death for each year of life. CONCLUSION: Heart disease and Diabetes mellitus were the most frequent comorbidities, but did not indicate an increased risk of death from SARS-CoV-2 infection. Age, sex, sore throat, dyspnea, respiratory discomfort, O2 saturation <95%, neurological disease, pneumopathy, immunodeficiency, and kidney disease were significantly associated with risk of death from COVID-19.""",
    """OBJECTIVE: To compare maternal and perinatal outcomes between randomized trials and observational studies in which conservative management was performed for more than 48 h in patients with early-onset severe preeclampsia. METHODOLOGY: We searched PubMed, LILACS, Cochrane and Google Scholar. The studies were divided in two groups: randomized and observational studies, from 1990 to 2018 that included patients with severe preeclampsia before 34 weeks of gestation with pregnancy prolongation &#8805;48 h but that did not include fetal growth restriction or HELLP syndrome at the beginning. The main variables recorded were maternal and perinatal complications. MAIN RESULTS: Forty-four studies met the inclusion criteria, and 5 of these were randomized. The average pregnancy prolongation was 9 days, with no difference between groups. Maternal complications were significantly more common in observational studies, RR = 0.71, 95% CI (0.54-0.93), p = .009. Perinatal complications were also significantly more common in observational studies (RR = 0.89, 95% CI (0.80-0.98), p = .01) at the expense of stillbirth and neonatal deaths. The percentages of cesarean sections were significantly higher in randomized studies, RR = 1.54, 95% CI (1.46-1.64). There were 2 maternal deaths, both in observational studies. CONCLUSION: Observational studies in which conservative management of early-onset preeclampsia is performed and do not include patients with fetal growth restriction or patients with HELLP syndrome and where at least 2 days of pregnancy prolongation is achieved are associated with significantly more maternal and perinatal complications.""",
    """Background:In recent years, an integrated programme of vector control, screening and treatment of gambiense human African trypanosomiasis (gHAT) infections has led to a rapid decline in cases in the Mandoul disease focus of Chad. In this study, we assess whether elimination of transmission has already been achieved in the region despite low-level case reporting, quantify the role of intensified interventions in transmission reduction, and predict the trajectory of gHAT in Mandoul for the next decade under a range of control scenarios. Method: We utilise human case data (2000-2019) to update a previous model of transmission of gHAT in Mandoul. We also test the updated model, which now has refined assumptions on diagnostic specificity of the current algorithm and an improved fitting method, via a data censoring approach.Results:We conclude that passive detection rates have increased due to improvements in diagnostic availability in fixed health facilities since 2015, by 2.1-fold for stage 1 detection, and by 1.5-fold for stage 2. We find that whilst the diagnostic algorithm for active screening is estimated to be highly specific (99.93%, 95% CI: 99.91-99.95%), the high screening level and limited remaining infection means that some recently reported cases might be false positives, especially the ones that were not parasitologically confirmed. We also find that the focus-wide tsetse vector reduction estimated through model fitting (99.1%, 95% CI: 96.1-99.6%) is comparable to the very high reduction previously measured by the decline in catches of tsetse from monitoring traps. In line with previous results, the model suggests that transmission was likely interrupted in 2015 as a result of intensified interventions. Conclusions: We recommend that additional confirmatory testing is performed in Mandoul in order that the endgame can be carefully monitored now that infection levels are so low. More specific measurement of cases would better inform when it is safe to stop active screening and vector control.""",
    """Microanatomy of the vast majority of human organs at birth is characterized by marked differences as compared to adult organs, regarding their architecture and the cell types detectable at histology. In preterm neonates, these differences are even more evident, due to the lower level of organ maturation and to ongoing cell differentiation. One of the most remarkable finding in preterm tissues is the presence of huge amounts of stem/progenitor cells in multiple organs, including kidney, brain, heart, adrenals, and lungs. In other organs, such as liver, the completely different burden of cell types in preterm infants is mainly related to the different function of the liver during gestation, mainly focused on hematopoiesis, a function that is taken by bone marrow after birth. Our preliminary studies showed that the antigens expressed by stem/progenitors differ significantly from one organ to the next. Moreover, within each developing human tissue, reactivity for different stem cell markers also changes during gestation, according with the multiple differentiation steps encountered by each progenitor during development. A better knowledge of stem/progenitor cells of preterms will allow neonatologists to boost preterm organ maturation, favoring the differentiation of the multiple cells types that characterize each organ in at term neonates.""",
]

#### Example: 1

In [13]:
input_data = {
    "input_text": covid19_articles, 
    "template": "summarization", 
    'max_new_tokens': 1024,
    'temperature': 0.2
}

# The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.
data = invoke_realtime_endpoint(input_data)
pd.DataFrame(data)

,response
0,"This study analyzed the clinical characteristics of older Brazilian adults (60+ years) infected with COVID-19 between August 16 and December 26, 2020. The study found that:\n\n* 48% of patients admitted to the ICU with COVID-19 died.\n* Factors associated with an increased risk of death included:\n * Immunodeficiency (188% increased risk)\n * Kidney disease (166% increased risk)\n * Neurological disease (103% increased risk)\n * Dyspnea (86% increased risk)\n * Pneumopathy (55% increased risk)\n * O2 saturation <95% (53% increased risk)\n * Respiratory discomfort (49% increased risk)\n * Age (36% increased risk per year of life)\n * Sore throat (31% increased risk)\n * Male sex (50% increased risk)\n* Heart disease and diabetes mellitus were common comorbidities but did not increase the risk of death from COVID-19.\n\nThese findings highlight the importance of considering age, sex, and specific comorbidities when assessing the risk of death from COVID-19 in older adults."
1,"This study compared the outcomes of conservative management of early-onset severe preeclampsia (high blood pressure during pregnancy) in randomized trials and observational studies. The study included 44 studies from 1990 to 2018, with 5 being randomized trials. The results showed that:\n\n* Maternal complications were more common in observational studies.\n* Perinatal complications (stillbirth and neonatal deaths) were also more common in observational studies.\n* Cesarean sections were more common in randomized trials.\n* There were 2 maternal deaths, both in observational studies.\n\nOverall, the study found that observational studies had more maternal and perinatal complications compared to randomized trials when conservative management was used for early-onset severe preeclampsia."
2,"The document discusses the progress made in controlling gambiense human African trypanosomiasis (gHAT) in the Mandoul region of Chad. The study used data from 2000 to 2019 to update a model of gHAT transmission in the region. The results suggest that:\n\n1. Passive detection rates have increased due to improved diagnostic availability in fixed health facilities since 2015.\n2. The diagnostic algorithm for active screening is highly specific (99.93%), but some recently reported cases may be false positives due to the high screening level and limited remaining infection.\n3. The focus-wide tsetse vector reduction is estimated to be 99.1%, which is comparable to the reduction measured by the decline in catches of tsetse from monitoring traps.\n4. Transmission was likely interrupted in 2015 due to intensified interventions.\n\nThe study concludes that additional confirmatory testing is needed to carefully monitor the endgame and determine when it is safe to stop active screening and vector control."
3,"The document discusses the differences in the microanatomy of human organs at birth compared to adult organs, particularly in preterm neonates. Key points include:\n\n1. **Organ Architecture and Cell Types**: The architecture and cell types of human organs at birth differ significantly from those in adult organs. These differences are more pronounced in preterm neonates due to lower organ maturation and ongoing cell differentiation.\n\n2. **Presence of Stem/Progenitor Cells**: Preterm tissues contain large amounts of stem/progenitor cells in various organs, including the kidney, brain, heart, adrenals, and lungs. These cells play a crucial role in organ development and maturation.\n\n3. **Organ-Specific Stem Cell Markers**: The antigens expressed by stem/progenitor cells vary significantly between organs. Within each developing tissue, the reactivity for different stem cell markers changes during gestation, reflecting the multiple differentiation steps encountered by each progenitor during development.\n\n4. **Implications for Neonatology**: Understanding stem/progenitor cells in preterm neonates can help neonatologists promote organ maturation and favor the diffe

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [16]:
import json
import os

# JSON file names
validation_json_file_name1 = "input1.json"
validation_json_file_name2 = "input2.json"

# JSONL file names
validation_jsonl_file_name1 = "input1.jsonl"
validation_jsonl_file_name2 = "input2.jsonl"


# JSON paths
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

# JSONL paths
validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"


def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )


In [17]:
def create_jsonl(records, max_new_tokens=None, temperature=None, repetition_penalty=None, template=None):
    if isinstance(records, str):
        records = [records]

    json_records = []

    for text in records:
        record = {
            "input_text": text,
            **{key: value for key, value in {
                "max_new_tokens": max_new_tokens,
                "temperature": temperature,
                "repetition_penalty": repetition_penalty,
                "template": template
            }.items() if value is not None}
        }

        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

# Example data for JSONL files
input_jsonl_data1 = create_jsonl(open_book_qa, template ="open_book_qa")
input_jsonl_data2 = create_jsonl(covid19_articles, template ="summarization")

# Example data for JSON files
input_json_data1 = json.dumps({"input_text": open_book_qa, "template":"open_book_qa"})
input_json_data2 = json.dumps({"input_text": covid19_articles, "template":"summarization"})


# Write and upload JSONL data
write_and_upload_to_s3(input_jsonl_data1, f"{validation_jsonl_file_name1}")
write_and_upload_to_s3(input_jsonl_data2, f"{validation_jsonl_file_name2}")

# Write and upload JSON data
write_and_upload_to_s3(input_json_data1, f"{validation_json_file_name1}")
write_and_upload_to_s3(input_json_data2,  f"{validation_json_file_name2}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
config = {
    'MAX_NEW_TOKENS': '1024',
    'TEMPERATURE': '0.2'
}

transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path,
    env=config
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [22]:
from urllib.parse import urlparse

def retrieve_json_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)


#### Example 1: Question Answering

In [23]:
# open_book_qa
retrieve_json_output_from_s3(validation_json_file_name1)

,response
0,The patient is worried about potential long-term side effects from radiation therapy.
1,"The patient experienced sharp pain in his lower back, followed by radiating symptoms down his right buttock and the lateral part of his leg, crossing his knee. These symptoms progressively worsened over time."
2,The primary issue reported by the patient is the recent worsening of her hypertension.


#### Example 2: Summarization

In [24]:
retrieve_json_output_from_s3(validation_json_file_name2)

,response
0,"The document discusses a study on the clinical characteristics of older Brazilian adults infected with COVID-19 and the factors that increase the risk of death. The study analyzed 1,544 confirmed cases of COVID-19 in patients aged 60 or older in Brazil between August 16 and December 26, 2020. The results showed that 48% of patients admitted to the ICU with COVID-19 died. The study found that the following factors were associated with an increased risk of death:\n\n* Age (5% increase in risk per year of life)\n* Sex (male)\n* Sore throat\n* Dyspnea (shortness of breath)\n* Respiratory discomfort\n* O2 saturation <95%\n* Neurological disease\n* Pneumopathy (lung disease)\n* Immunodeficiency\n* Kidney disease\n\nThe study also found that heart disease and diabetes mellitus were the most common comorbidities, but they did not increase the risk of death from COVID-19."
1,"This study compared the outcomes of conservative management of early-onset severe preeclampsia in pregnant women before 34 weeks of gestation between randomized trials and observational studies. The study found that:\n\n* Maternal complications were more common in observational studies (RR = 0.71, 95% CI (0.54-0.93), p = .009).\n* Perinatal complications were also more common in observational studies (RR = 0.89, 95% CI (0.80-0.98), p = .01), mainly due to stillbirth and neonatal deaths.\n* Cesarean sections were more common in randomized studies (RR = 1.54, 95% CI (1.46-1.64)).\n* There were 2 maternal deaths, both in observational studies.\n\nOverall, the study suggests that conservative management of early-onset severe preeclampsia in observational studies is associated with more maternal and perinatal complications compared to randomized trials."
2,"The document discusses a study on the transmission of gambiense human African trypanosomiasis (gHAT) in the Mandoul region of Chad. The study used human case data from 2000 to 2019 to update a previous model of gHAT transmission. The results suggest that:\n\n* Passive detection rates have increased due to improved diagnostic availability in fixed health facilities since 2015.\n* The diagnostic algorithm for active screening is highly specific (99.93%), but some recently reported cases may be false positives due to the high screening level and limited remaining infection.\n* The focus-wide tsetse vector reduction is estimated to be 99.1%, which is comparable to the reduction measured by the decline in catches of tsetse from monitoring traps.\n* Transmission was likely interrupted in 2015 due to intensified interventions.\n\nThe study concludes that additional confirmatory testing is needed to carefully monitor the endgame and determine when it is safe to stop active screening and vector control."
3,"The document discusses the differences in the microanatomy of human organs at birth compared to adult organs. It highlights that preterm neonates have even more pronounced differences due to lower organ maturation and ongoing cell differentiation. The presence of a large number of stem/progenitor cells in various organs, such as the kidney, brain, heart, adrenals, and lungs, is a notable finding in preterm tissues. The liver, however, has a different cell composition in preterm infants due to its primary function of hematopoiesis during gestation, which is later taken over by the bone marrow after birth. The document also mentions that the antigens expressed by stem/progenitor cells vary significantly between organs and change during gestation as the cells undergo differentiation. Understanding stem/progenitor cells in preterm infants can help neonatologists promote organ maturation and differentiation, ultimately leading to healthier outcomes."


### JSON Lines

In [ ]:
config = {
    'MAX_NEW_TOKENS': '1024',
    'TEMPERATURE': '0.2'
}

transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path,
    env=config
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [26]:
from urllib.parse import urlparse

def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

#### Example 1: Question Answering

In [27]:
# open_book_qa
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name1)

{"response": "The patient is worried about potential long-term side effects from radiation therapy."}
{"response": "The patient experienced sharp pain in his lower back, followed by radiating symptoms down his right buttock and the lateral part of his leg, crossing his knee. These symptoms progressively worsened over time."}
{"response": "The primary issue reported by the patient is the recent worsening of her hypertension."}


#### Example 2: Summarization

In [28]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name2)

{"response": "The document discusses a study on the clinical characteristics of older Brazilian adults infected with COVID-19 and the factors that increase the risk of death. The study analyzed 1,544 confirmed cases of COVID-19 in patients aged 60 or older in Brazil between August 16 and December 26, 2020. The results showed that 48% of patients admitted to the ICU with COVID-19 died. The study found that the following factors were associated with an increased risk of death:\n\n*   Age (5% increase in risk per year of life)\n*   Sex (male)\n*   Sore throat\n*   Dyspnea (shortness of breath)\n*   Respiratory discomfort\n*   O2 saturation <95%\n*   Neurological disease\n*   Pneumopathy (lung disease)\n*   Immunodeficiency\n*   Kidney disease\n\nThe study also found that heart disease and diabetes mellitus were the most common comorbidities, but they did not increase the risk of death from COVID-19."}
{"response": "This study compared the outcomes of conservative management of early-onset

In [29]:
model.delete_model()

INFO:sagemaker:Deleting model with name: jsl-medical-summarization-qa-70b-2024-11-07-17-44-02-522


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

